## Approximating atan2

In [ ]:
import math
import numpy
import matplotlib.pyplot

In [ ]:
px = numpy.array([0, 0.40, 0.95, 1.60, 2.50, 3.8,  5.00, 7.00, 15, 25])
py = numpy.array([0, 0.39, 0.76, 1.03, 1.19, 1.32,  1.38, 1.44, 1.51, 1.54])

for x, y in zip(px, py):
    print(f"    {{ .x = {int(x * 1024)}, .y = {int(y * 180 / numpy.pi * 8)} }},")

# Python implementation of pbio_math_atan2_positive.
def atan_single_pos(y, x):
    ratio = abs(y / x)
    for i in range(len(px)-1):
        if ratio < px[i+1]:
            return py[i] + (ratio - px[i]) * (py[i+1] - py[i]) / (px[i+1] - px[i]) 
    return py[-1]

# Python implementation of pbio_math_atan2.
def atan2_single(y, x):
    atan = atan_single_pos(abs(y), abs(x))
    if (x > 0 and y < 0) or (x < 0 and y > 0):
        atan *= -1
    if x > 0:
        return atan
    if atan > 0:
        return atan - numpy.pi
    else:
        return atan + numpy.pi

# Numpy-like variant that can deal with arrays.
def atan2(y, x):
    return numpy.array([atan2_single(b, a) for b, a in zip(y, x)])

# Get two axes in one figure.
figure, axes = matplotlib.pyplot.subplots(nrows=1, ncols=2,figsize=(20, 10))
atan_axis, error_axis = axes

ratios = numpy.linspace(-40, 40, num=10000)
atan_real = numpy.arctan(ratios)
atan_appr = numpy.array([atan2_single(r, 1) for r in ratios])

# Plot the atan data.
atan_axis.plot(ratios, atan_real, label="Real")
atan_axis.plot(ratios, atan_appr, label="Approximation")
atan_axis.plot(px, py, label="points", marker="o")
atan_axis.set_ylabel("atan (rad)")

# Plot error data
error_axis.plot(ratios, numpy.degrees(atan_appr - atan_real), label="Error")
error_axis.set_ylabel("Error (deg)")

# Show grid, legend, and fix bounds.
for axis in (atan_axis, error_axis):
    axis.grid(True)
    axis.legend()
    axis.set_xlim(-20, 20)
    axis.set_ylim(-2, 2)
    axis.set_xlabel("ratio (y/x)")

## Atan2 tilt error

In [ ]:
# Sample data for full roll turn
angle = numpy.linspace(-numpy.pi, numpy.pi, num=1000)
angle_deg = numpy.degrees(angle)
accel_z = 9810 * numpy.cos(angle)
accel_y = 9810 * numpy.sin(angle)

# Result if atan2 is available.
roll = numpy.degrees(numpy.arctan2(accel_y, accel_z))
roll_simple = numpy.degrees(atan2(accel_y, accel_z))

# Get two axes in one figure.
figure, (angle_axis, error_axis) = matplotlib.pyplot.subplots(nrows=1, ncols=2,figsize=(15, 10))

# Plot angle data
angle_axis.plot(angle_deg, roll, label="roll via atan2")
angle_axis.plot(angle_deg, roll_simple, label="roll approximation")

# Plot angle data
error_axis.plot(angle_deg, roll-roll_simple, label="Error (deg)")
error_axis.set_ylim(-3, 3)

# Show grid, legend, and fix bounds.
for axis in (angle_axis, error_axis):
    axis.grid(True)
    axis.legend()
    axis.set_xlim(angle_deg[0], angle_deg[-1])
    axis.set_xlabel("Angle")